In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def get_dormitory_meals():
    """
    군산대학교 생활관 식단표를 파싱하는 함수
    Returns:
        dict: 아침, 점심, 저녁 별 데이터프레임
    """
    url = "https://www.kunsan.ac.kr/dormi/index.kunsan?menuCd=DOM_000000704006003000"
    
    # 웹페이지 가져오기
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 식단표 테이블 찾기 - class나 id를 통해 더 정확하게 찾기
    tables = soup.find_all('table', class_='board-list')
    if not tables:
        raise Exception("식단표를 찾을 수 없습니다.")
    
    table = tables[0]
    rows = table.find_all('tr')
    if not rows:
        raise Exception("식단 데이터를 찾을 수 없습니다.")
    
    # 결과를 저장할 딕셔너리
    meals = {
        '아침': [],
        '점심': [],
        '저녁': []
    }
    
    # 요일 리스트
    days = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    
    # 식단 종류별로 처리
    meal_types = list(meals.keys())
    
    # 디버깅을 위한 출력
    print(f"Found {len(rows)} rows in table")
    
    for row in rows:
        columns = row.find_all('td')
        if len(columns) == 7:  # 요일별 데이터가 있는 행만 처리
            # 어떤 식사인지 확인 (아침/점심/저녁)
            meal_index = rows.index(row)
            if meal_index >= len(meal_types):
                continue
                
            meal_type = meal_types[meal_index]
            
            # 각 요일별 메뉴 저장
            for day_idx, col in enumerate(columns):
                menu = col.get_text(strip=True)
                if not menu:
                    menu = "식단 정보 없음"
                meals[meal_type].append({
                    '요일': days[day_idx],
                    '메뉴': menu
                })
    
    # 데이터프레임으로 변환
    meal_dfs = {}
    for meal_type, data in meals.items():
        if data:  # 데이터가 있는 경우만 데이터프레임 생성
            meal_dfs[meal_type] = pd.DataFrame(data)
        else:
            meal_dfs[meal_type] = pd.DataFrame(columns=['요일', '메뉴'])
    
    return meal_dfs

def print_today_menu():
    """
    오늘의 식단을 출력하는 함수
    """
    try:
        meals = get_dormitory_meals()
        today = datetime.now().weekday()  # 0 = 월요일, 6 = 일요일
        
        print(f"=== 오늘의 식단 ===")
        for meal_type, df in meals.items():
            if not df.empty and len(df) > today:
                menu = df.iloc[today]['메뉴']
                print(f"\n{meal_type}:")
                print(menu)
            else:
                print(f"\n{meal_type}:")
                print("식단 정보가 없습니다.")
    except Exception as e:
        print(f"식단 정보를 가져오는 중 오류가 발생했습니다: {str(e)}")

def print_weekly_menu():
    """
    이번 주 전체 식단을 출력하는 함수
    """
    try:
        meals = get_dormitory_meals()
        print("=== 이번 주 식단 ===")
        for meal_type, df in meals.items():
            print(f"\n[ {meal_type} ]")
            if not df.empty:
                for _, row in df.iterrows():
                    print(f"{row['요일']}: {row['메뉴']}")
            else:
                print("식단 정보가 없습니다.")
    except Exception as e:
        print(f"식단 정보를 가져오는 중 오류가 발생했습니다: {str(e)}")

if __name__ == "__main__":
    print_weekly_menu()  # 전체 주간 식단 출력

식단 정보를 가져오는 중 오류가 발생했습니다: 식단표를 찾을 수 없습니다.


In [3]:
response = requests.get("https://www.kunsan.ac.kr/dormi/index.kunsan?menuCd=DOM_000000704006003000")
response.encoding = 'utf-8'
print(response.text)










	









<!DOCTYPE html>
<html lang="ko">
<head>
<meta charset="utf-8" />
<meta http-equiv="Cache-Control" content="no-cache"><!--230914 추가-->
<link rel="stylesheet" type="text/css" href="/rfc3/user/domain/www.kunsan.ac.kr~dormi.80/0/8.css" /><!-- 공통 base -->
<link rel="stylesheet" type="text/css" href="/rfc3/user/domain/www.kunsan.ac.kr~dormi.80/0/9.css" /><!-- 공통 board -->
<link rel="stylesheet" type="text/css" href="/rfc3/user/domain/www.kunsan.ac.kr~dormi.80/0/10.css" /><!--공통 bottom -->
<link rel="stylesheet" type="text/css" href="/rfc3/user/domain/www.kunsan.ac.kr~dormi.80/0/18.css" /><!--공통 gnb_top -->

<link rel="stylesheet" type="text/css" href="/rfc3/user/domain/www.kunsan.ac.kr~dormi.80/0/320.css" /><!-- layout -->
<link rel="stylesheet" type="text/css" href="/rfc3/user/domain/www.kunsan.ac.kr~dormi.80/0/318.css" /><!-- gnb -->
<link rel="stylesheet" type="text/css" href="/rfc3/user/domain/www.kunsan.ac.kr~dormi.80/0/319.css" /><!-- lnb -->
<link rel="stylesheet" 

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime
import time

def get_dormitory_meals():
    """
    군산대학교 생활관 식단표를 파싱하는 함수
    Returns:
        dict: 아침, 점심, 저녁 별 데이터프레임
    """
    # Chrome 옵션 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 브라우저 창을 띄우지 않음
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    # WebDriver 초기화
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    try:
        # 웹페이지 로드
        url = "https://www.kunsan.ac.kr/dormi/index.kunsan?menuCd=DOM_000000704006003000"
        driver.get(url)
        
        # 테이블이 로드될 때까지 대기 (최대 10초)
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ctable01"))
        )
        
        # 요일 및 날짜 정보 가져오기
        headers = table.find_elements(By.TAG_NAME, "th")[1:]  # 첫 번째 th는 빈 칸이므로 제외
        days = []
        dates = []
        for header in headers:
            text = header.text.split('\n')[0]  # '월', '화' 등
            date = header.find_element(By.TAG_NAME, "span").text  # '11.4' 등
            days.append(text)
            dates.append(date)
        
        # 식사 종류별 데이터 가져오기
        tbody = table.find_element(By.TAG_NAME, "tbody")
        rows = tbody.find_elements(By.TAG_NAME, "tr")
        
        meals = {}
        meal_types = ['아침', '점심', '저녁']
        
        for row in rows:
            # 식사 종류 (아침/점심/저녁)
            meal_type = row.find_element(By.TAG_NAME, "th").text
            
            # 각 요일별 메뉴
            menu_cells = row.find_elements(By.TAG_NAME, "td")
            daily_menus = []
            
            for i, cell in enumerate(menu_cells):
                menus = cell.text.split('\n')
                menus = [menu.strip() for menu in menus if menu.strip()]
                if not menus or menus[0] == '미운영':
                    menu_str = "미운영"
                else:
                    menu_str = '\n'.join(menus)
                
                daily_menus.append({
                    '요일': days[i],
                    '날짜': dates[i],
                    '메뉴': menu_str
                })
            
            meals[meal_type] = pd.DataFrame(daily_menus)
        
        return meals
        
    finally:
        driver.quit()

def print_today_menu():
    """
    오늘의 식단을 출력하는 함수
    """
    try:
        meals = get_dormitory_meals()
        today = datetime.now().weekday()  # 0 = 월요일, 6 = 일요일
        
        print(f"\n=== 오늘의 식단 ===")
        for meal_type, df in meals.items():
            if not df.empty and len(df) > today:
                menu = df.iloc[today]['메뉴']
                print(f"\n[ {meal_type} ]")
                print(menu)
            else:
                print(f"\n[ {meal_type} ]")
                print("식단 정보가 없습니다.")
    except Exception as e:
        print(f"식단 정보를 가져오는 중 오류가 발생했습니다: {str(e)}")

def print_weekly_menu():
    """
    이번 주 전체 식단을 출력하는 함수
    """
    try:
        meals = get_dormitory_meals()
        print("\n=== 이번 주 식단 ===")
        
        for meal_type, df in meals.items():
            print(f"\n[ {meal_type} ]")
            if not df.empty:
                for _, row in df.iterrows():
                    print(f"\n{row['요일']}({row['날짜']}):")
                    print(f"{row['메뉴']}")
            else:
                print("식단 정보가 없습니다.")
    except Exception as e:
        print(f"식단 정보를 가져오는 중 오류가 발생했습니다: {str(e)}")

def get_specific_day_menu(weekday):
    """
    특정 요일의 식단을 출력하는 함수
    
    Args:
        weekday (str): '월', '화', '수', '목', '금', '토', '일' 중 하나
    """
    try:
        meals = get_dormitory_meals()
        print(f"\n=== {weekday}요일 식단 ===")
        
        for meal_type, df in meals.items():
            if not df.empty:
                day_menu = df[df['요일'] == weekday]
                if not day_menu.empty:
                    print(f"\n[ {meal_type} ]")
                    print(f"{day_menu.iloc[0]['메뉴']}")
            else:
                print(f"\n[ {meal_type} ]")
                print("식단 정보가 없습니다.")
    except Exception as e:
        print(f"식단 정보를 가져오는 중 오류가 발생했습니다: {str(e)}")

if __name__ == "__main__":
    # print_weekly_menu()        # 전체 주간 식단
    # print_today_menu()         # 오늘의 식단
    get_specific_day_menu('월')  # 특정 요일 식단

식단 정보를 가져오는 중 오류가 발생했습니다: __init__() got an unexpected keyword argument 'service'


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

def get_dormitory_meals():
    # 구형 ChromeDriver 설정으로 변경
    driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=webdriver.ChromeOptions().add_argument('--headless'))

    try:
        url = "https://www.kunsan.ac.kr/dormi/index.kunsan?menuCd=DOM_000000704006003000"
        driver.get(url)
        
        # 테이블이 로드될 때까지 대기
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ctable01"))
        )
        
        headers = table.find_elements(By.TAG_NAME, "th")[1:]  # 첫 번째 th는 빈 칸이므로 제외
        days, dates = [], []
        for header in headers:
            day_text = header.text.split('\n')[0]
            date_text = header.find_element(By.TAG_NAME, "span").text
            days.append(day_text)
            dates.append(date_text)
        
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        meals = {"아침": [], "점심": [], "저녁": []}
        meal_types = ["아침", "점심", "저녁"]
        
        for row, meal_type in zip(rows[1:], meal_types):
            cells = row.find_elements(By.TAG_NAME, "td")
            daily_menus = []
            
            for i, cell in enumerate(cells):
                menu_text = cell.text.strip()
                daily_menus.append({
                    "요일": days[i],
                    "날짜": dates[i],
                    "메뉴": menu_text or "미운영"
                })
            
            meals[meal_type] = pd.DataFrame(daily_menus)

        return meals

    finally:
        driver.quit()

def print_today_menu():
    meals = get_dormitory_meals()
    today_idx = datetime.now().weekday()
    print(f"\n=== 오늘의 식단 ===")
    for meal_type, df in meals.items():
        if not df.empty and len(df) > today_idx:
            menu = df.iloc[today_idx]['메뉴']
            print(f"\n[ {meal_type} ]")
            print(menu)
        else:
            print(f"\n[ {meal_type} ]")
            print("식단 정보가 없습니다.")

if __name__ == "__main__":
    print_today_menu()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span"}
  (Session info: chrome=130.0.6723.92)


In [9]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

def get_dormitory_meals():
    driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=webdriver.ChromeOptions().add_argument('--headless'))

    try:
        url = "https://www.kunsan.ac.kr/dormi/index.kunsan?menuCd=DOM_000000704006003000"
        driver.get(url)
        
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ctable01"))
        )
        
        headers = table.find_elements(By.TAG_NAME, "th")[1:]
        days, dates = [], []

        for header in headers:
            day_text = header.text.split('\n')[0]
            # 'span' 요소가 있는지 확인하여 처리
            span_elements = header.find_elements(By.TAG_NAME, "span")
            date_text = span_elements[0].text if span_elements else "미정"  # span 요소가 없으면 '미정'으로 설정
            days.append(day_text)
            dates.append(date_text)
        
        rows = table.find_elements(By.TAG_NAME, "tr")
        meals = {"아침": [], "점심": [], "저녁": []}
        meal_types = ["아침", "점심", "저녁"]
        
        for row, meal_type in zip(rows[1:], meal_types):
            cells = row.find_elements(By.TAG_NAME, "td")
            daily_menus = []
            
            for i, cell in enumerate(cells):
                menu_text = cell.text.strip()
                daily_menus.append({
                    "요일": days[i],
                    "날짜": dates[i],
                    "메뉴": menu_text or "미운영"
                })
            
            meals[meal_type] = pd.DataFrame(daily_menus)

        return meals

    finally:
        driver.quit()

def print_today_menu():
    meals = get_dormitory_meals()
    today_idx = datetime.now().weekday()
    print(f"\n=== 오늘의 식단 ===")
    for meal_type, df in meals.items():
        if not df.empty and len(df) > today_idx:
            menu = df.iloc[today_idx]['메뉴']
            print(f"\n[ {meal_type} ]")
            print(menu)
        else:
            print(f"\n[ {meal_type} ]")
            print("식단 정보가 없습니다.")

if __name__ == "__main__":
    print_today_menu()



=== 오늘의 식단 ===

[ 아침 ]
11:40~13:30

[ 점심 ]
12:00~13:00

[ 저녁 ]
12:00~13:00


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime

def get_dormitory_meals():
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=webdriver.ChromeOptions().add_argument('--headless'))

    try:
        url = "https://www.kunsan.ac.kr/dormi/index.kunsan?menuCd=DOM_000000704006003000"
        driver.get(url)
        
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ctable01"))
        )
        
        headers = table.find_elements(By.TAG_NAME, "th")[1:]
        days, dates = [], []

        for header in headers:
            day_text = header.text.split('\n')[0]
            span_elements = header.find_elements(By.TAG_NAME, "span")
            date_text = span_elements[0].text if span_elements else "미정"  # span 요소가 없으면 '미정'으로 설정
            days.append(day_text)
            dates.append(date_text)
        
        rows = table.find_elements(By.TAG_NAME, "tr")
        meals = {"아침": [], "점심": [], "저녁": []}
        meal_types = ["아침", "점심", "저녁"]
        
        for row, meal_type in zip(rows[1:], meal_types):
            cells = row.find_elements(By.TAG_NAME, "td")
            daily_menus = []
            
            for i, cell in enumerate(cells):
                # 메뉴 텍스트가 없는 경우에 대비하여 예외 처리
                menu_text = cell.text.strip() if cell.text.strip() else "미운영"
                daily_menus.append({
                    "요일": days[i],
                    "날짜": dates[i],
                    "메뉴": menu_text
                })
            
            meals[meal_type] = pd.DataFrame(daily_menus)

        return meals

    finally:
        driver.quit()

def print_today_menu():
    meals = get_dormitory_meals()
    today_idx = datetime.now().weekday()
    print(f"\n=== 오늘의 식단 ===")
    for meal_type, df in meals.items():
        if not df.empty and len(df) > today_idx:
            menu = df.iloc[today_idx]['메뉴']
            print(f"\n[ {meal_type} ]")
            print(menu)
        else:
            print(f"\n[ {meal_type} ]")
            print("식단 정보가 없습니다.")

if __name__ == "__main__":
    print_today_menu()



=== 오늘의 식단 ===

[ 아침 ]
11:40~13:30

[ 점심 ]
12:00~13:00

[ 저녁 ]
12:00~13:00


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime

def get_dormitory_meals():
    # Chrome 드라이버 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

    try:
        url = "https://www.kunsan.ac.kr/dormi/index.kunsan?menuCd=DOM_000000704006003000"
        driver.get(url)
        
        # 테이블 요소를 대기하면서 로드
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ctable01"))
        )
        
        # 요일과 날짜 가져오기
        headers = table.find_elements(By.TAG_NAME, "th")[1:]
        days, dates = [], []

        for header in headers:
            day_text = header.text.split('\n')[0]
            span_elements = header.find_elements(By.TAG_NAME, "span")
            date_text = span_elements[0].text if span_elements else "미정"  # span 요소가 없으면 '미정'으로 설정
            days.append(day_text)
            dates.append(date_text)
        
        # 각 끼니별 메뉴 데이터 저장
        rows = table.find_elements(By.TAG_NAME, "tr")
        meals = {"아침": [], "점심": [], "저녁": []}
        meal_types = ["아침", "점심", "저녁"]
        
        for row, meal_type in zip(rows[1:], meal_types):
            cells = row.find_elements(By.TAG_NAME, "td")
            daily_menus = []
            
            for i, cell in enumerate(cells):
                # 각 셀의 텍스트를 추출하여 비어있으면 "미운영" 처리
                menu_text = cell.text.strip() if cell.text.strip() else "미운영"
                print(f"[DEBUG] {meal_type} - {days[i]} {dates[i]} 메뉴: {menu_text}")  # 디버그용 출력
                daily_menus.append({
                    "요일": days[i],
                    "날짜": dates[i],
                    "메뉴": menu_text
                })
            
            meals[meal_type] = pd.DataFrame(daily_menus)

        return meals

    finally:
        driver.quit()

def print_today_menu():
    meals = get_dormitory_meals()
    today_idx = datetime.now().weekday()
    print(f"\n=== 오늘의 식단 ===")
    for meal_type, df in meals.items():
        if not df.empty and len(df) > today_idx:
            menu = df.iloc[today_idx]['메뉴']
            print(f"\n[ {meal_type} ]")
            print(menu)
        else:
            print(f"\n[ {meal_type} ]")
            print("식단 정보가 없습니다.")

if __name__ == "__main__":
    print_today_menu()


[DEBUG] 아침 - 아침 미정 메뉴: 07:40~09:00
[DEBUG] 아침 - 점심 미정 메뉴: 11:40~13:30
[DEBUG] 아침 - 저녁 미정 메뉴: 17:40~19:00
[DEBUG] 점심 - 아침 미정 메뉴: (미운영)
[DEBUG] 점심 - 점심 미정 메뉴: 12:00~13:00
[DEBUG] 점심 - 저녁 미정 메뉴: 17:40~18:40
[DEBUG] 저녁 - 아침 미정 메뉴: 08:00~09:00
[DEBUG] 저녁 - 점심 미정 메뉴: 12:00~13:00
[DEBUG] 저녁 - 저녁 미정 메뉴: 17:40~18:40

=== 오늘의 식단 ===

[ 아침 ]
11:40~13:30

[ 점심 ]
12:00~13:00

[ 저녁 ]
12:00~13:00


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re
from datetime import datetime

def get_dormitory_meals():
    # Chrome 드라이버 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

    try:
        url = "https://www.kunsan.ac.kr/dormi/index.kunsan?menuCd=DOM_000000704006003000"
        driver.get(url)
        
        # 테이블 요소 로드 대기
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ctable01"))
        )
        
        # 요일과 날짜 가져오기
        headers = table.find_elements(By.TAG_NAME, "th")[1:]
        days, dates = [], []

        for header in headers:
            day_text = header.text.split('\n')[0]
            span_elements = header.find_elements(By.TAG_NAME, "span")
            date_text = span_elements[0].text if span_elements else "미정"  # span 요소가 없으면 '미정'으로 설정
            days.append(day_text)
            dates.append(date_text)
        
        # 각 끼니별 메뉴 데이터
        rows = table.find_elements(By.TAG_NAME, "tr")
        meals = {"아침": [], "점심": [], "저녁": []}
        meal_types = ["아침", "점심", "저녁"]
        
        for row, meal_type in zip(rows[1:], meal_types):
            cells = row.find_elements(By.TAG_NAME, "td")
            daily_menus = []
            
            for i, cell in enumerate(cells):
                # 메뉴 텍스트만 필터링 (시간 패턴 제거)
                raw_text = cell.text.strip()
                menu_text = re.sub(r'\d{1,2}:\d{2}~\d{1,2}:\d{2}', '', raw_text).strip()  # 시간 패턴 제거
                if not menu_text:
                    menu_text = "미운영"  # 메뉴가 없을 경우 '미운영'
                print(f"[DEBUG] {meal_type} - {days[i]} {dates[i]} 메뉴: {menu_text}")  # 디버그용 출력
                daily_menus.append({
                    "요일": days[i],
                    "날짜": dates[i],
                    "메뉴": menu_text
                })
            
            meals[meal_type] = pd.DataFrame(daily_menus)

        return meals

    finally:
        driver.quit()

def print_today_menu():
    meals = get_dormitory_meals()
    today_idx = datetime.now().weekday()
    print(f"\n=== 오늘의 식단 ===")
    for meal_type, df in meals.items():
        if not df.empty and len(df) > today_idx:
            menu = df.iloc[today_idx]['메뉴']
            print(f"\n[ {meal_type} ]")
            print(menu)
        else:
            print(f"\n[ {meal_type} ]")
            print("식단 정보가 없습니다.")

if __name__ == "__main__":
    print_today_menu()


[DEBUG] 아침 - 아침 미정 메뉴: 미운영
[DEBUG] 아침 - 점심 미정 메뉴: 미운영
[DEBUG] 아침 - 저녁 미정 메뉴: 미운영
[DEBUG] 점심 - 아침 미정 메뉴: (미운영)
[DEBUG] 점심 - 점심 미정 메뉴: 미운영
[DEBUG] 점심 - 저녁 미정 메뉴: 미운영
[DEBUG] 저녁 - 아침 미정 메뉴: 미운영
[DEBUG] 저녁 - 점심 미정 메뉴: 미운영
[DEBUG] 저녁 - 저녁 미정 메뉴: 미운영

=== 오늘의 식단 ===

[ 아침 ]
미운영

[ 점심 ]
미운영

[ 저녁 ]
미운영
